# BUILD THE NEURAL NETWORK
- 인공신경망은 **layer** 와 **module** 로 구성됨
- **torch.nn** 네임스페이스는 사용자 정의 네트워크를 만들기 위한 거의 모든 building block 을 제공함
- 모든 모듈은 **nn.Module** 을 상속함
- 인공신경망은 그 자체로 하나의 모듈임

- 여기선 FashonMNIST 데이터셋 이미지를 분류하기 위한 인공신경망을 구성해보겠음

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

---
# 1. Get Device for Training
- **GPU** 와 같은 하드웨어 가속기가 있다면 사용하는 것이 좋음
- **torch.cuda** 를 통해 GPU 사용이 가능한지 확인 후 가능하다면 사용, 안되면 CPU 사용

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


---
# 2. Define the Class
- ```nn.Module``` 을 상속하여 사용자 정의 인공신경망을 정의
- ```__init__``` 생성자에서 해당 인공신경망에서 사용할 layer 를 초기화함
- ```forward``` 메서드에서 입력 데이터에 대한 처리 연산을 수행

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

- ```NeuralNetwork``` 의 인스턴스를 만들고 ```device``` 로 이동시킴

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


- 해당 모델을 사용하려면 입력 데이터를 넣으면 됨
- 모델에 입력 데이터를 넣으면 자동으로 ```forward``` 메서드가 실행됨 (직접 호출하지 않도록 주의!)

In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)

print(f"Logits: {logits}\n")
print(f"Predicted Probabilty: {pred_prob}\n")
print(f"Predicted class: {y_pred}")

Logits: tensor([[0.0507, 0.0445, 0.0000, 0.0991, 0.0094, 0.0000, 0.0687, 0.0000, 0.0000,
         0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

Predicted Probabilty: tensor([[0.1023, 0.1017, 0.0973, 0.1074, 0.0982, 0.0973, 0.1042, 0.0973, 0.0973,
         0.0973]], device='cuda:0', grad_fn=<SoftmaxBackward>)

Predicted class: tensor([3], device='cuda:0')


---
# 3. Model Parameters
- 인공신경망 내부 대부분의 layer 들은 *parameterized* 되어 있음 (학습으로 최적화되는 weight 와 bias 로 구성)
- ```nn.Module``` 을 상속하면 자동으로 사용자가 정의한 모델 객체 내부에서 정의된 모든 필드를 추적해줌
- ```parameters()``` 또는 ```named_parameters()``` 메서드를 이용해 모든 파라미터에 접근할 수 있음

In [10]:
print("Model structure: ", model, '\n\n')

for name, param in model.named_parameters():
    print(f"Layer: {name}")
    print(f"Size: {param.size()}")
    print(f"Values: {param[: 2]}\n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight
Size: torch.Size([512, 784])
Values: tensor([[-0.0002, -0.0347, -0.0204,  ..., -0.0232, -0.0222,  0.0346],
        [-0.0180,  0.0095,  0.0122,  ..., -0.0014,  0.0238,  0.0011]],
       device='cuda:0', grad_fn=<SliceBackward>)

Layer: linear_relu_stack.0.bias
Size: torch.Size([512])
Values: tensor([-0.0326,  0.0038], device='cuda:0', grad_fn=<SliceBackward>)

Layer: linear_relu_stack.2.weight
Size: torch.Size([512, 512])
Values: tensor([[-0.0251, -0.0434, -0.0070,  ..., -0.0108, -0.0187, -0.0208],
        [ 0.0067,  0.0121, -0.0187,  ...,  0.0286,  0.0399, -0.0023]],
       device='cuda:0', grad_fn=<Sli